In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import os
os.listdir()

['.ipynb_checkpoints', 'K-Mean Clustrning.ipynb', 'Online Retail.xlsx']

In [25]:
df= pd.read_excel('Online Retail.xlsx')

In [26]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [27]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [33]:
# --- Step 1: Handle Missing Values ---
df= df.dropna()

In [29]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [34]:
# --- Step 2: Remove Duplicate Entries ---
df = df.drop_duplicates()

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [32]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,406829.000000,406829,406829.000000,406829.000000
mean,12.061303,2011-07-10 16:30:57.879207424,3.460471,15287.690570
min,-80995.000000,2010-12-01 08:26:00,0.000000,12346.000000
25%,2.000000,2011-04-06 15:02:00,1.250000,13953.000000
50%,5.000000,2011-07-31 11:48:00,1.950000,15152.000000
75%,12.000000,2011-10-20 13:06:00,3.750000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,248.693370,NaN,69.315162,1713.600303


In [47]:
# Convert CustomerID to integer type
df['CustomerID'] = df['CustomerID'].astype(int)

In [42]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif

In [43]:
# --- Step 3: Select Numerical Columns for Standardization ---
numeric_cols = ['Quantity', 'UnitPrice']

In [44]:
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [49]:
# Encode categorical variables
lb_encoder_country = LabelEncoder()
df['Country'] = lb_encoder_country.fit_transform(df['Country'].astype(str))

lb_encoder_stock = LabelEncoder()
df['StockCode'] = lb_encoder_stock.fit_transform(df['StockCode'].astype(str))

In [50]:
# -- Feature Engineering ---
# Create new features
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df['InvoiceYear'] = df['InvoiceDate'].dt.year
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceDay'] = df['InvoiceDate'].dt.day
df['InvoiceHour'] = df['InvoiceDate'].dt.hour

In [52]:
# Create a target column: High value order
df['HighValueOrder'] = (df['TotalPrice'] > df['TotalPrice'].median()).astype(int)

# Drop irrelevant columns
df = df.drop(['InvoiceNo', 'Description'], axis=1)

In [53]:
# predicting HighValueOrder
X = df.drop('HighValueOrder', axis=1)
y = df['HighValueOrder']

In [56]:
X = X.select_dtypes(include=['int64', 'float64'])

selector = SelectKBest(score_func=f_classif, k=5)
X_new = selector.fit_transform(X, y)

In [57]:
selected_features = X.columns[selector.get_support()]

In [59]:
print(selected_features)

Index(['StockCode', 'Quantity', 'CustomerID', 'Country', 'TotalPrice'], dtype='object')




 
**Understand**        Displays head, info, describe, and missing values             
**Clean**             Removes missing/duplicate rows, fixes data types              
**Transform**         Standardizes numerical values, encodes categories             
**Feature Engineer**  Creates new features (TotalPrice, Date parts, HighValueOrder)    
 **Feature Select**    Identifies most relevant features using ANOVA F-test      


In [ ]:
df.to_excel("OnlineRetail_Final_Preprocessed.xlsx", index=False)